In [1]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import matplotlib.dates as mdates
import csv

In [2]:
# GROUND TRUTH

In [32]:
video_num = 11 # CHANGE
video_durations = [73*60 + 29, 33*60 + 52, 40*60, 31*60 + 13, 30*60 + 25]
video_num_frames = [110225, 50800, 60000, 46825, 45600]

directory_gt = 'Ground Truth Timelines/tool_video_' + str(video_num) + '.txt'
with open(directory_gt) as f:
    f.next() # skip header line
    reader = csv.reader(f, delimiter = '\t')
    data = [(col1, col2, col3, col4, col5, col6, col7, col8) for col1, col2, col3, col4, col5, col6, col7, col8 in reader]

duration = video_durations[video_num - 11]
num_frames = video_num_frames[video_num - 11]
frame_increment = 25 # every 25 frames were used in testing
fps = num_frames/float(duration)
frame_length = duration/float(num_frames) # each frame accounts for this much time (seconds)

In [33]:
# when each tool is being used
grasper_times_gt = []
# grasper2_times = []
bipolar_times_gt = []
hook_times_gt = []
scissors_times_gt = []
clipper_times_gt = []
irrigator_times_gt = []
specimenbag_times_gt = []

# how long each tool is used for
grasper_total_gt = 0
# grasper2_total = 0
bipolar_total_gt = 0
hook_total_gt = 0
scissors_total_gt = 0
clipper_total_gt = 0
irrigator_total_gt = 0
specimenbag_total_gt = 0

In [34]:
m = len(data) # number of rows
n = len(data[0]) # number of columns

for i in range(0, m):
    
    frame_num = int(data[i][0])
    pos = frame_num/frame_increment * frame_length * frame_increment
    
    for j in range(1, n):
        if int(data[i][j]) == 1:
            if j == 1:
                grasper_times_gt.append((pos,frame_length*frame_increment))
                grasper_total_gt = grasper_total_gt + frame_length*frame_increment
            elif j == 2:
                bipolar_times_gt.append((pos,frame_length*frame_increment))
                bipolar_total_gt = bipolar_total_gt + frame_length*frame_increment
            elif j == 3:
                hook_times_gt.append((pos,frame_length*frame_increment))
                hook_total_gt = hook_total_gt + frame_length*frame_increment
            elif j == 4:
                scissors_times_gt.append((pos,frame_length*frame_increment))
                scissors_total_gt = scissors_total_gt + frame_length*frame_increment
            elif j == 5:
                clipper_times_gt.append((pos,frame_length*frame_increment))
                clipper_total_gt = clipper_total_gt + frame_length*frame_increment
            elif j == 6:
                irrigator_times_gt.append((pos,frame_length*frame_increment))
                irrigator_total_gt = irrigator_total_gt + frame_length*frame_increment
            elif j == 7:    
                specimenbag_times_gt.append((pos,frame_length*frame_increment))
                specimenbag_total_gt = specimenbag_total_gt + frame_length*frame_increment

In [35]:
# PREDICTED

In [36]:
# directory = 'tool_video_' + str(video_num) + '_image_test.csv'
# if video_num == 15:
#     directory = '0.8_threshold/tool_video_' + str(video_num) + '_image_test.csv'

# improved 7-26-17 model
directory = '../CSV Files 7-26-17 Model 81.8 mAP/0.8 Threshold/tool_video_' + str(video_num) + '_image_test.csv'
df = pd.read_csv(directory)

In [37]:
class_column = df['class']
list_tools = class_column[0:7]

frame_column = df['frame']
c1_column = df['coord_ 1']
c6_column = df['coord_ 6']
c11_column = df['coord_11']

In [38]:
# when each tool is being used
# with moving average (ma)
grasper_times_ma = []
grasper2_times_ma = []
bipolar_times_ma = []
hook_times_ma = []
scissors_times_ma = []
clipper_times_ma = []
irrigator_times_ma = []
specimenbag_times_ma = []

# how long each tool is used for
# with moving average (ma)
grasper_total_ma = 0
grasper2_total_ma = 0
bipolar_total_ma = 0
hook_total_ma = 0
scissors_total_ma = 0
clipper_total_ma = 0
irrigator_total_ma = 0
specimenbag_total_ma = 0

In [39]:
n = sum(1 for line in open(directory)) # number of lines in csv

for i in range(0, n-1): # rows of csv: 0 to n-2
    if math.isnan(c1_column[i]) == False: # if there is a coordinate value, i.e. there is a tool
        
        if video_num == 15:
            if i > 6: # 7 rows for frame 000000.jpg
                frame_num = frame_column[i]
                frame_num = int(frame_num.lstrip("0")[4:][:-4]) # remove "v15_", leading zeros, and ".jpg" at the end of the frame name, convert str to int
            else: # 0 <= i <= 6
                frame_num = 0
        else:
            if i > 6: # 7 rows for frame 000000.jpg
                frame_num = frame_column[i]
                frame_num = int(frame_num.lstrip("0")[:-4]) # remove leading zeros and ".jpg" at the end of the frame name, convert str to int
            else:
                frame_num = 0 # first frame starts at 000000.jpg; have to include if-else to get a non-empty first frame_num
        pos = frame_num/frame_increment * frame_length * frame_increment
        
        if class_column[i] == 'Grasper':
            
            # moving average, check three frames before and three frames after; if 3/6 contain this tool, add onto timeline
            grasper_count_ma = 0
            if i >= 21 and i <= (n-2-21):
                if math.isnan(c1_column[i+7]) == False: # grasper in subsequent frame
                    grasper_count_ma = grasper_count_ma + 1
                if math.isnan(c1_column[i+14]) == False:
                    grasper_count_ma = grasper_count_ma + 1
                if math.isnan(c1_column[i+21]) == False:
                    grasper_count_ma = grasper_count_ma + 1
                if math.isnan(c1_column[i-7]) == False: # grasper in previous frame
                    grasper_count_ma = grasper_count_ma + 1
                if math.isnan(c1_column[i-14]) == False:
                    grasper_count_ma = grasper_count_ma + 1
                if math.isnan(c1_column[i-21]) == False:
                    grasper_count_ma = grasper_count_ma + 1
            # if grasper_count_ma >= 3:
            if grasper_count_ma >= 2: # improved 7-26-17 model
                grasper_times_ma.append((pos,frame_length*frame_increment))
                grasper_total_ma = grasper_total_ma + frame_length*frame_increment
            
            # ma for second grasper
            if math.isnan(c6_column[i]) == False:
                grasper2_count_ma = 0
                if i >= 21 and i <= (n-2-21):
                    if math.isnan(c6_column[i+7]) == False:
                        grasper2_count_ma = grasper2_count_ma + 1
                    if math.isnan(c6_column[i+14]) == False:
                        grasper2_count_ma = grasper2_count_ma + 1
                    if math.isnan(c6_column[i+21]) == False:
                        grasper2_count_ma = grasper2_count_ma + 1
                    if math.isnan(c6_column[i-7]) == False:
                        grasper2_count_ma = grasper2_count_ma + 1
                    if math.isnan(c6_column[i-14]) == False:
                        grasper2_count_ma = grasper2_count_ma + 1
                    if math.isnan(c6_column[i-21]) == False:
                        grasper2_count_ma = grasper2_count_ma + 1
                if grasper2_count_ma >= 3:
                    grasper2_times_ma.append((pos,frame_length*frame_increment))
                    grasper2_total_ma = grasper2_total_ma + frame_length*frame_increment
            '''
            grasper_times.append((pos,frame_length*frame_increment)) # frame_length*frame_increment = how long the bar should be drawn to represent one frame's duration
            grasper_total = grasper_total + frame_length*frame_increment
            if math.isnan(c6_column[i]) == False: # another grasper in this frame
                grasper2_times.append((pos,frame_length*frame_increment))
                grasper2_total = grasper2_total + frame_length*frame_increment
            '''

        elif class_column[i] == 'Bipolar':
            
            bipolar_count_ma = 0
            if i >= 21 and i <= (n-2-21):
                if math.isnan(c1_column[i+7]) == False:
                    bipolar_count_ma = bipolar_count_ma + 1
                if math.isnan(c1_column[i+14]) == False:
                    bipolar_count_ma = bipolar_count_ma + 1
                if math.isnan(c1_column[i+21]) == False:
                    bipolar_count_ma = bipolar_count_ma + 1
                if math.isnan(c1_column[i-7]) == False:
                    bipolar_count_ma = bipolar_count_ma + 1
                if math.isnan(c1_column[i-14]) == False:
                    bipolar_count_ma = bipolar_count_ma + 1
                if math.isnan(c1_column[i-21]) == False:
                    bipolar_count_ma = bipolar_count_ma + 1
            # if bipolar_count_ma >= 3:
            if bipolar_count_ma >= 2: # improved 7-26-17 model
                bipolar_times_ma.append((pos,frame_length*frame_increment))
                bipolar_total_ma = bipolar_total_ma + frame_length*frame_increment
                
            # bipolar_times.append((pos,frame_length*frame_increment))
            # bipolar_total = bipolar_total + frame_length*frame_increment
            
        elif class_column[i] == 'Hook':
            
            hook_count_ma = 0
            if i >= 21 and i <= (n-2-21):
                if math.isnan(c1_column[i+7]) == False:
                    hook_count_ma = hook_count_ma + 1
                if math.isnan(c1_column[i+14]) == False:
                    hook_count_ma = hook_count_ma + 1
                if math.isnan(c1_column[i+21]) == False:
                    hook_count_ma = hook_count_ma + 1
                if math.isnan(c1_column[i-7]) == False:
                    hook_count_ma = hook_count_ma + 1
                if math.isnan(c1_column[i-14]) == False:
                    hook_count_ma = hook_count_ma + 1
                if math.isnan(c1_column[i-21]) == False:
                    hook_count_ma = hook_count_ma + 1
            # if hook_count_ma >= 3:
            if hook_count_ma >= 2: # improved 7-26-17 model
                hook_times_ma.append((pos,frame_length*frame_increment))
                hook_total_ma = hook_total_ma + frame_length*frame_increment
                
            # hook_times.append((pos,frame_length*frame_increment))
            # hook_total = hook_total + frame_length*frame_increment
            
        elif class_column[i] == 'Scissors':
            
            scissors_count_ma = 0
            if i >= 21 and i <= (n-2-21):
                if math.isnan(c1_column[i+7]) == False:
                    scissors_count_ma = scissors_count_ma + 1
                if math.isnan(c1_column[i+14]) == False:
                    scissors_count_ma = scissors_count_ma + 1
                if math.isnan(c1_column[i+21]) == False:
                    scissors_count_ma = scissors_count_ma + 1
                if math.isnan(c1_column[i-7]) == False:
                    scissors_count_ma = scissors_count_ma + 1
                if math.isnan(c1_column[i-14]) == False:
                    scissors_count_ma = scissors_count_ma + 1
                if math.isnan(c1_column[i-21]) == False:
                    scissors_count_ma = scissors_count_ma + 1
            # if scissors_count_ma >= 3:
            if scissors_count_ma >= 2: # improved 7-26-17 model
                scissors_times_ma.append((pos,frame_length*frame_increment))
                scissors_total_ma = scissors_total_ma + frame_length*frame_increment
            
            # scissors_times.append((pos,frame_length*frame_increment))
            # scissors_total = scissors_total + frame_length*frame_increment
        
        elif class_column[i] == 'Clipper':
            
            clipper_count_ma = 0
            if i >= 21 and i <= (n-2-21):
                if math.isnan(c1_column[i+7]) == False:
                    clipper_count_ma = clipper_count_ma + 1
                if math.isnan(c1_column[i+14]) == False:
                    clipper_count_ma = clipper_count_ma + 1
                if math.isnan(c1_column[i+21]) == False:
                    clipper_count_ma = clipper_count_ma + 1
                if math.isnan(c1_column[i-7]) == False:
                    clipper_count_ma = clipper_count_ma + 1
                if math.isnan(c1_column[i-14]) == False:
                    clipper_count_ma = clipper_count_ma + 1
                if math.isnan(c1_column[i-21]) == False:
                    clipper_count_ma = clipper_count_ma + 1
            # if clipper_count_ma >= 3:
            if clipper_count_ma >= 2: # improved 7-26-17 model
                clipper_times_ma.append((pos,frame_length*frame_increment))
                clipper_total_ma = clipper_total_ma + frame_length*frame_increment
                
            # clipper_times.append((pos,frame_length*frame_increment))
            # clipper_total = clipper_total + frame_length*frame_increment
            
        elif class_column[i] == 'Irrigator':
            
            irrigator_count_ma = 0
            if i >= 21 and i <= (n-2-21):
                if math.isnan(c1_column[i+7]) == False:
                    irrigator_count_ma = irrigator_count_ma + 1
                if math.isnan(c1_column[i+14]) == False:
                    irrigator_count_ma = irrigator_count_ma + 1
                if math.isnan(c1_column[i+21]) == False:
                    irrigator_count_ma = irrigator_count_ma + 1
                if math.isnan(c1_column[i-7]) == False:
                    irrigator_count_ma = irrigator_count_ma + 1
                if math.isnan(c1_column[i-14]) == False:
                    irrigator_count_ma = irrigator_count_ma + 1
                if math.isnan(c1_column[i-21]) == False:
                    irrigator_count_ma = irrigator_count_ma + 1
            # if irrigator_count_ma >= 3:
            if irrigator_count_ma >= 2: # improved 7-26-17 model
                irrigator_times_ma.append((pos,frame_length*frame_increment))
                irrigator_total_ma = irrigator_total_ma + frame_length*frame_increment
            
            # irrigator_times.append((pos,frame_length*frame_increment))
            # irrigator_total = irrigator_total + frame_length*frame_increment
                
        elif class_column[i] == 'SpecimenBag':
            
            specimenbag_count_ma = 0
            if i >= 21 and i <= (n-2-21):
                if math.isnan(c1_column[i+7]) == False:
                    specimenbag_count_ma = specimenbag_count_ma + 1
                if math.isnan(c1_column[i+14]) == False:
                    specimenbag_count_ma = specimenbag_count_ma + 1
                if math.isnan(c1_column[i+21]) == False:
                    specimenbag_count_ma = specimenbag_count_ma + 1
                if math.isnan(c1_column[i-7]) == False:
                    specimenbag_count_ma = specimenbag_count_ma + 1
                if math.isnan(c1_column[i-14]) == False:
                    specimenbag_count_ma = specimenbag_count_ma + 1
                if math.isnan(c1_column[i-21]) == False:
                    specimenbag_count_ma = specimenbag_count_ma + 1
            # if specimenbag_count_ma >= 3:
            if specimenbag_count_ma >= 2: # improved 7-26-17 model
                specimenbag_times_ma.append((pos,frame_length*frame_increment))
                specimenbag_total_ma = specimenbag_total_ma + frame_length*frame_increment
            
            # specimenbag_times.append((pos,frame_length*frame_increment))
            # specimenbag_total = specimenbag_total + frame_length*frame_increment

In [41]:
fig, ax = plt.subplots()

width = 2.5

color = '#0000ff' # blue
color_gt = '#008000' # green for gt

ax.broken_barh(specimenbag_times_gt, (5, width), color = '#008000', edgecolors = 'none')
ax.broken_barh(specimenbag_times_ma, (5+width, width), color = '#0000ff', edgecolors = 'none')

ax.broken_barh(irrigator_times_gt, (12.5, width), color = '#008000', edgecolors = 'none')
ax.broken_barh(irrigator_times_ma, (12.5+width, width), color = '#0000ff', edgecolors = 'none')

ax.broken_barh(clipper_times_gt, (20, width), color = '#008000', edgecolors = 'none')
ax.broken_barh(clipper_times_ma, (20+width, width), color = '#0000ff', edgecolors = 'none')

ax.broken_barh(scissors_times_gt, (27.5, width), color = '#008000', edgecolors = 'none')
ax.broken_barh(scissors_times_ma, (27.5+width, width), color = '#0000ff', edgecolors = 'none')

ax.broken_barh(hook_times_gt, (35, width), color = '#008000', edgecolors = 'none')
ax.broken_barh(hook_times_ma, (35+width, width), color = '#0000ff', edgecolors = 'none')

ax.broken_barh(bipolar_times_gt, (42.5, width), color = '#008000', edgecolors = 'none')
ax.broken_barh(bipolar_times_ma, (42.5+width, width), color = '#0000ff', edgecolors = 'none')

# ax.broken_barh(grasper2_times, (35, width), edgecolors = color)

ax.broken_barh(grasper_times_gt, (50, width), color = '#008000', edgecolors = 'none')
ax.broken_barh(grasper_times_ma, (50+width, width), color = '#0000ff', edgecolors = 'none')

x = 7.5

space_between_bars = 5
ax.set_ylim(0, 7.375+6*x+7.5)
ax.set_xlim(0, duration)

font = {'fontname':'Arial'}


ax.set_yticks([7.375, 7.375+x, 7.375+2*x, 7.375+3*x, 7.375+4*x, 7.375+5*x, 7.375+6*x])
ax.set_yticklabels('g' 'f' 'e' 'd' 'c' 'b' 'a')
# ax.set_yticklabels(['Specimen Bag', 'Irrigator', 'Clipper', 'Scissors', 'Hook', 'Bipolar', 'Grasper'], **font)


# x-axis ticks and labels
video_minutes_rounded = [70, 30, 40, 30, 30] # CHANGE
xticks = [0, video_minutes_rounded[video_num - 11]*60/5, video_minutes_rounded[video_num - 11]*60/5*2, video_minutes_rounded[video_num - 11]*60/5*3, video_minutes_rounded[video_num - 11]*60/5*4, video_minutes_rounded[video_num - 11]*60]
ax.set_xticks(xticks) # five evenly spaced ticks
if video_num == 11:
    ax.set_xticklabels(['00:00', '14:00', '28:00', '42:00', '56:00', '01:10:00'], **font)
elif video_num == 12 or 14 or 15:
    ax.set_xticklabels(['00:00', '06:00', '12:00', '18:00', '24:00', '30:00'], **font)
else:
    ax.set_xticklabels(['00:00', '08:00', '16:00', '24:00', '32:00', '40:00'], **font)

# plt.xticks(np.arange(0, duration, duration-1), **font)
# plt.xticks(**font)

plt.rc('xtick', labelsize = 24) # fontsize of the tick labels
plt.rc('ytick', labelsize = 24)
# ax.set_title('Testing Video 1: Instrument Usage During the Surgery')

# change x-axis to H:M:S

fig.tight_layout()

plt.show()
# fig.savefig('Ground Truth Timelines/' + str(video_num) + '_comparison.svg', format='svg', dpi=1200)
# improved 7-26-17 model
fig.savefig('../CSV Files 7-26-17 Model 81.8 mAP/0.8 Threshold/Timelines/WACV/' + str(video_num) + '_comparison_smaller_ma.svg', format='svg', dpi=1200)

In [12]:
[(73*60 + 29)/60, (33*60 + 52)/60, (40*60)/60, (31*60 + 13)/60, (30*60 + 25)/60]

[73, 33, 40, 31, 30]